# Parte 1

## Imports

- **numpy**: 
- **pandas**: 
- **nltk**: Para POS tagging, tokenização de sentenças e demais itens inerentes a NLP
- **emoji**: Para converter emojis no texto para suas respectivas descriçrões
- _emot_: Considerar o uso para extrair emojis e emoticons do texto
- **re**: Para encontrar padrões por meio de expressões regulares

In [1]:
import numpy as np
import pandas as pd
import nltk
import emoji
import re

## Leitura dos dados

Lendo os dados do arquivo **banzeiro.json** em duas listas de objetos, que serão representados por `dicts`. Uma lista guardará os dados com os emojis substituídos pelas suas respectivas descrições e a outra, o conteúdo original.

In [2]:
posts = pd.read_json("banzeiro.json")
posts.head()

,key,datetime,img_urls,caption,comments
0,https://www.instagram.com/p/B2mMB_OHbfI/,2019-09-19 14:47:48+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,https://www.instagram.com/p/B2ef0osHDik/,2019-09-16 15:06:49+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,https://www.instagram.com/p/B2ZXEGSn1di/,2019-09-14 15:14:05+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,https://www.instagram.com/p/B2UpaJ5HLWg/,2019-09-12 19:23:34+00:00,[],"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,https://www.instagram.com/p/B2PCM4Qn9mD/,2019-09-10 14:59:22+00:00,[https://z-p42-instagram.fpll2-1.fna.fbcdn.net...,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."


Verificando a estrutura dos dados, é possível perceber que existem colunas que não contém texto excrito por usuários, que uma publicação não possui legenda (`caption`) e que nem todas as instâncias possuem um vetor de comentários a ser processado (`comments`).

In [3]:
posts.count()

key         778
datetime    778
img_urls    778
caption     777
comments    606
dtype: int64

# Parte 2

## Limpeza de dados

Algumas operações de limpeza dos dados deverão ser realizadas, considerando os seguintes pontos:

- Algumas colunas não são relevantes no processamento de texto
- Comentários que apenas mencionam outros usuários não expressam uma opinião
- Publicações sem legenda não deixam claro a que se endereça a opinião
- Publicações sem comentários não possuem opiniões a serem mineradas


### Removendo colunas sem importância

In [4]:
posts = posts.drop(["key", "datetime", "img_urls"], axis=1)

In [5]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."
774,"Pôr do sol em São Gabriel da Cachoeira, por Fe...",NaN
775,Viaje com os sabores e combinações exóticas do...,NaN
776,Petit gateau amazônico.\nVenha saborear essa m...,"[{'author': 'victorer83', 'comment': '@marinay..."


### Removendo linhas onde caption ou comments são nulos

In [6]:
posts = posts.dropna()

In [7]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
765,"Desbrave novos sabores, embale seu paladar e d...","[{'author': 'mellothz', 'comment': 'Estive no ..."
767,Pastel de pato. Deliciosamente incomparável. U...,"[{'author': 'chriscalderaro', 'comment': 'Eu a..."
771,O chef Felipe Schaedler foi convidado para par...,"[{'author': 'thiagosuico', 'comment': '@felipe..."
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."


### Removendo comentários que somente mencionam outros usuários

In [8]:
for index, row in posts.iterrows():
    for comment in row["comments"]:
        if re.search("^(@[a-z0-9_.]*\s*)+$", comment["comment"]):
            posts.at[index, "comments"].remove(comment)
posts = posts.dropna()

In [9]:
posts

,caption,comments
0,Já provou os nossos Chás Gelados?\nAs três com...,"[{'author': 'audreycaminha', 'comment': 'Delic..."
1,Uma das entradas exclusivas do Banzeiro Manaus...,"[{'author': 'karenmabel', 'comment': 'é sensac..."
2,“Todos os dias penso no compromisso que tenho ...,"[{'author': '59cmlc', 'comment': 'Estive no Ba..."
3,"Felipe Schaedler, Cidadão Amazonense!\n\nAssis...","[{'author': 'ricfeitoza', 'comment': '👏🏽👏🏽👏🏽👏🏽..."
4,"Tanga verde de folhas, corpo claro de Cacau e ...","[{'author': 'ginapsic', 'comment': 'Delicia'},..."
...,...,...
765,"Desbrave novos sabores, embale seu paladar e d...","[{'author': 'mellothz', 'comment': 'Estive no ..."
767,Pastel de pato. Deliciosamente incomparável. U...,"[{'author': 'chriscalderaro', 'comment': 'Eu a..."
771,O chef Felipe Schaedler foi convidado para par...,"[{'author': 'thiagosuico', 'comment': '@felipe..."
773,Totalmente diferente de tudo o que você já pro...,"[{'author': 'gilson_monteiro', 'comment': 'O q..."


# Parte 3

## Convertendo o dataframe para dict

Com a parte de limpeza dos dados finalizada e antes do processamento realizado pelo NLTK, os dados serão representados em forma de **dict**.

In [10]:
posts = posts.to_dict('records')


## Tokenizando sentenças e Strings

Fazendo o uso do **nltk**, serão tokenizadas as sentenças e palavras do dataset. As sentenças serão quebradas com o tokenizador **Punkt** treinado para o Português, e as palavras, quebrando nos espaços em branco.

In [12]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/miguel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

for p in posts:
    p["caption"] = sent_tokenizer.tokenize(p["caption"])
    for index in range(len(p["caption"])):
        p["caption"][index] = nltk.tokenize.word_tokenize(p["caption"][index])
    for c in p['comments']:
        c["comment"] = nltk.tokenize.word_tokenize(c["comment"], language="portuguese")